## 1. Requeriments

In [1]:
# import os
# import torch
# os.environ['TORCH'] = torch.__version__
# print(torch.__version__)

# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# # !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cpu.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
# !pip install -q captum

## 2. Import libraries

In [2]:
import torch
from torch_geometric.data import Data
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import preprocessing
import random

## 3. Graph building

### 3.1. Gene matrix

In [3]:
genes = pd.read_csv('..\..\Data\PPT-Ohmnet\mRCC_big_pool\mrcc_protein_matrix_2405_genes_732_nodes.csv')
Y = genes.Y

genes = genes.iloc[:,1:733] 
genes

,ABCB1,ABCB6,ABCG2,ABL2,ACADM,ACD,ACE,ACE2,ACHE,ACTA2,...,XPO1,XPR1,XRCC6,YAP1,YBX1,YBX3,YY1,ZBTB17,ZHX2,ZMIZ1
0,36.716012,33.867624,29.676682,32.862716,35.061520,31.801427,31.988036,30.172489,29.912204,37.060160,...,34.62167,32.34863,35.24634,33.97677,37.51551,35.75761,33.53412,31.72285,33.23150,32.38760
1,34.174577,32.703008,31.318871,33.061603,35.908450,31.878071,33.015718,33.634947,26.076400,37.751704,...,35.01883,32.98690,35.91455,34.75610,36.67356,35.85355,34.44291,31.63512,32.63491,33.78434
2,31.225510,34.522351,30.612181,32.658413,34.370223,31.425178,31.714695,26.858096,27.589284,36.813611,...,35.39763,31.74722,36.05801,34.48484,36.41164,35.24518,35.14050,31.99899,34.31359,32.95630
3,32.988641,33.059035,31.973489,33.014431,35.824161,31.829645,32.916062,29.565514,28.143610,39.627694,...,35.67365,33.18418,35.75676,35.27953,36.58061,36.19379,34.96911,31.28469,33.03073,33.88207
4,33.284849,31.488348,30.774368,33.793065,34.332987,32.020127,31.824945,28.767942,26.537023,38.329374,...,36.74827,32.83641,36.33015,34.50745,36.89459,34.95292,34.76483,33.40109,32.15993,33.35772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,33.838323,32.296025,30.740420,33.040376,34.981970,31.823358,32.748254,25.870831,27.355529,38.421074,...,35.25426,31.85605,35.53767,34.82946,37.20378,36.41314,34.69950,32.89423,33.07640,33.73563
177,31.895951,33.784466,29.347511,31.571621,34.865097,29.786302,29.720601,30.945513,23.149098,35.035788,...,36.17748,32.14064,35.50224,33.69401,36.55929,35.16855,34.89385,33.76237,31.38631,30.87456
178,33.842290,32.838072,29.862937,33.491465,34.379400,31.246333,33.114433,25.870831,28.926076,37.644991,...,35.19340,32.47980,35.22805,34.52318,36.62523,35.18621,34.88021,31.47358,31.68824,33.42545
179,34.715293,32.561403,29.292590,33.151894,34.802123,32.344355,33.543569,32.700981,29.967641,37.914176,...,35.92815,32.49879,34.81409,34.59853,34.81241,35.19760,34.37022,31.75344,33.62427,33.09948


### 3.2. Graph edges

In [4]:
path ='../../Data/PPT-Ohmnet/mRCC_big_pool/network_edges_mrcc_2405_genes_732_nodes.tsv'
data = pd.read_csv(path, delimiter='\t')
edge_index1=data['EPO'].to_numpy()
edge_index2=data['GATA3'].to_numpy()

In [5]:
edge_index = np.concatenate((edge_index1, edge_index2))

In [6]:
le = preprocessing.LabelEncoder()
le.fit(edge_index)
len(list(le.classes_))

732

In [7]:
edge_index1 = le.transform(edge_index1)
edge_index2 = le.transform(edge_index2)

In [8]:
edge_index = [edge_index1]+[edge_index2]
edge_index = np.array(edge_index)

In [9]:
edge_index

array([[213, 213, 213, ..., 604, 605, 213],
       [299, 263, 214, ..., 606, 606, 264]])

In [10]:
edge_index = torch.tensor(edge_index, dtype=torch.int64)
edge_index

tensor([[213, 213, 213,  ..., 604, 605, 213],
        [299, 263, 214,  ..., 606, 606, 264]])

### 3.3. Create dataset

In [11]:
list_data_0=[]
list_data_1=[]

for g in range(len(genes)):
  b=[]
  for i in genes.iloc[g].to_numpy():
    a=[]
    a.append(i*10)
    b.append(a)
  x = torch.tensor([b], dtype=torch.float).reshape([-1])
  edge_index = edge_index
  y = torch.tensor([Y.iloc[g]], dtype=torch.float).reshape([-1, 1])
  data = Data(x=x, edge_index=edge_index, y=y)
  if y == 0:
    list_data_0.append(data)
  else:
    list_data_1.append(data)

print(list_data_0[0])

Data(x=[732], edge_index=[2, 6990], y=[1, 1])


In [12]:
len(list_data_1)

96

### 3.4. Patient sample

In [13]:
data = list_data_0[0]
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of charcateristics per node: {data.num_features}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of nodes: 732
Number of charcateristics per node: 1
Number of edges: 6990
Average node degree: 9.55
Has isolated nodes: False
Has self-loops: False
Is undirected: False


## 4. Graph training

### 4.1. Train-Test splitting

In [14]:
# torch.manual_seed(1255)
random.shuffle(list_data_0)
random.shuffle(list_data_1)

train_dataset = list_data_0[0:72]
test_dataset = list_data_0[72:86]
train_dataset = train_dataset + list_data_1[0:81]
test_dataset = test_dataset + list_data_1[81:97]
random.shuffle(train_dataset)
random.shuffle(test_dataset)
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')
cont = 0
cont1=0
for i in train_dataset:
    if i.y == 0:
        cont+=1
for i in test_dataset:
    if i.y == 0:
        cont1+=1
print("Negative cases from train: " + str(cont) + " of " + str(len(train_dataset)) + " = " + str(cont/len(train_dataset)))
print("Negative cases from test: " + str(cont1) + " of " + str(len(test_dataset)) + " = " + str(cont1/len(test_dataset)))
print("It should be 46.9")

Number of training graphs: 153
Number of test graphs: 28
Negative cases from train: 72 of 153 = 0.47058823529411764
Negative cases from test: 13 of 28 = 0.4642857142857143
It should be 46.9


In [15]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 32
DataBatch(x=[23424], edge_index=[2, 223680], y=[32, 1], batch=[23424], ptr=[33])

Step 2:
Number of graphs in the current batch: 32
DataBatch(x=[23424], edge_index=[2, 223680], y=[32, 1], batch=[23424], ptr=[33])

Step 3:
Number of graphs in the current batch: 32
DataBatch(x=[23424], edge_index=[2, 223680], y=[32, 1], batch=[23424], ptr=[33])

Step 4:
Number of graphs in the current batch: 32
DataBatch(x=[23424], edge_index=[2, 223680], y=[32, 1], batch=[23424], ptr=[33])

Step 5:
Number of graphs in the current batch: 25
DataBatch(x=[18300], edge_index=[2, 174750], y=[25, 1], batch=[18300], ptr=[26])



### 4.2. Training and testing the model

In [16]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GraphConv
from torch_geometric.nn import SAGPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch import nn

In [17]:
embed_dim = 732

class Net(torch.nn.Module):
    def __init__(self, dim):
        super(Net, self).__init__()
        self.dim = dim
        super(Net, self).__init__()
        self.conv1 = GraphConv(embed_dim, dim)
        self.pool1 = SAGPooling(dim, ratio=0.2)
        self.conv2 = GraphConv(dim, dim)
        self.pool2 = SAGPooling(dim, ratio=0.2)
        self.item_embedding = torch.nn.Embedding(num_embeddings=426, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(11712, 5000)
        self.lin2 = torch.nn.Linear(5000, 500)
        self.lin3 = torch.nn.Linear(500, 1)
        self.act1 = torch.nn.ReLU()
        print(self)

    def forward(self, x, edge_index, batch):
        x = torch.tensor(x).to(torch.int)
        x = self.item_embedding(x)
        x = x.squeeze(1)

        x = F.relu(self.conv1(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act1(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = torch.sigmoid(self.lin3(x)).squeeze(1)
        return x


In [18]:
def train(epoch):
    model.train()
    criterion = nn.BCELoss()
    loss_all = 0
    for data in train_loader:
        output = model(data.x, data.edge_index, data.batch)
        loss = criterion(output, data.y.squeeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_all += loss.item() * data.num_graphs

    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data
        output = model(data.x, data.edge_index, data.batch)
        for i in range(len(output)):
            if output[i]>0.5:
                output[i]=1
            else:
                output[i]=0
            if output[i]==data.y[i]:
                correct=correct+1
    # print("Correct: "+str(correct) +" of "+str(len(loader.dataset)))
    return correct / len(loader.dataset)

In [19]:
import matplotlib.pyplot as plt
test_avg = []
for i in range(5):
    model = Net(dim=5856)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    train_epoch=[]
    test_epoch=[]
    epoch = 1
    train_acc=0
    while train_acc < 0.9 and epoch < 50:
        loss = train(epoch)
        train_acc = test(train_loader)
        test_acc = test(test_loader)
        train_epoch.append(train_acc)
        test_epoch.append(test_acc)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
            f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
        epoch +=1

    plt.plot(train_epoch, color="red")
    plt.plot(test_epoch, color="blue")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    test_avg.append(test_acc)

print('Test accuracy: '+ str(np.array(test_avg).mean()))
print('Test stv: '+ str(np.array(test_avg).std()))

Net(
  (conv1): GraphConv(732, 5856)
  (pool1): SAGPooling(GraphConv, 5856, ratio=0.2, multiplier=1.0)
  (conv2): GraphConv(5856, 5856)
  (pool2): SAGPooling(GraphConv, 5856, ratio=0.2, multiplier=1.0)
  (item_embedding): Embedding(426, 732)
  (lin1): Linear(in_features=11712, out_features=5000, bias=True)
  (lin2): Linear(in_features=5000, out_features=500, bias=True)
  (lin3): Linear(in_features=500, out_features=1, bias=True)
  (act1): ReLU()
)


C:\Users\sandr\AppData\Local\Temp/ipykernel_17068/4117278113.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).to(torch.int)


Epoch: 001, Loss: 8.2440, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 002, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 003, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 004, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 005, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 006, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 007, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 008, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643
Epoch: 009, Loss: 52.9412, Train Acc: 0.4706, Test Acc: 0.4643


KeyboardInterrupt: 